In [1]:
from ground_data import GroundObservations
from LOCSS_data_structure import GaugeCollection
import importlib
import pandas as pd
import os
import datetime as dt
%load_ext autoreload
%autoreload 2

In [28]:
#Generate each lake in a folder
go=GroundObservations()
df_locss_podaac=go.get_locss('LOCSS', all_fds=True)

g_path='../data/locss_2_podaac/by_st'
f_exist=os.path.exists(g_path)
stations=df_locss_podaac['gauge_id'].unique()
df_locss_podaac.rename(columns={'name_x':'name_at_measurement', 'notes_x':'reading_notes', 'name_y':'lake_name',
                                       'notes_y':'installation_notes'}, inplace=True)
st_fields=['gauge_id','lake_name','date','height','is_bubble_level_okay','installation_notes', 'city','latitude','longitude','timezone',
           'unit','min_height','max_height','installation_date','initial_reading','reading_notes', 'updated_at']
df_locss_podaac=df_locss_podaac[st_fields].copy()
if not f_exist:
    os.makedirs(g_path)
for st in stations:
    st_path=g_path+'/'+st
    f_st_exist=os.path.exists(st_path)
    if not f_st_exist:
        os.makedirs(st_path)
    df=df_locss_podaac.loc[df_locss_podaac['gauge_id']==st]
    df.to_csv(st_path+'/LOCSS_'+st+'.csv')

In [25]:
df_locss_podaac.columns


Index(['gauge_id', 'lake_name', 'date', 'height', 'is_bubble_level_okay',
       'installation_notes', 'city', 'latitude', 'longitude', 'timezone',
       'unit', 'min_height', 'max_height', 'installation_date',
       'initial_reading', 'reading_notes', 'updated_at'],
      dtype='object')

In [33]:
#read with no merge for po.daac
dt_fields=['gauge_id','name_x','date','height','is_bubble_level_okay','notes_x']
st_fields=['gauge_id','name','city','latitude','longitude','timezone','unit','min_height','max_height','installation_date','initial_reading','notes', 'updated_at']

dir_ts='../data/readings_up_to_20220404.csv'
dir_loc='../data/gauges_up_to_20220404.csv'
id_fd='gauge_id'
date_fd='date'
time_fd='time'
height_fd='height'
df_locss=pd.read_csv(dir_ts)
df_coord_locss=pd.read_csv(dir_loc, sep=",")
gc=GaugeCollection()

df_locss_filtered=gc.filter_test_gages(df_locss,id_fd).copy()
df_locss_filtered=pd.merge(df_locss_filtered, df_coord_locss, on=id_fd)
df_locss_filtered=df_locss_filtered.loc[(df_locss_filtered[height_fd]>=df_locss_filtered['min_height'])&
                                        (df_locss_filtered[height_fd]<=df_locss_filtered['max_height'])].copy()
df_locss_filtered[date_fd]=pd.to_datetime((df_locss_filtered[date_fd].astype(str)+' '+df_locss_filtered[time_fd].astype(str)), 
                                                  format='%Y-%m-%d %H:%M:%S')


In [34]:
df_locss_filtered=df_locss_filtered[dt_fields]
df_locss_filtered.rename(columns={'name_x':'name', 'notes_x':'notes'}, inplace=True)
df_coord_locss=df_coord_locss[st_fields]
out_path='../data/locss_2_podaac/st_rd/'
f_exist=os.path.exists(out_path)
if not f_exist:
    os.makedirs(out_path)
df_locss_filtered.to_csv(out_path+'locss_readings_up_to_20220404.csv', index=False)
df_coord_locss.to_csv(out_path+'locss_gauges_up_to_20220404.csv', index=False)

In [67]:
#Test getting brasilian ts
import pandas as pd
import os


def _get_all_lakes(file_list, by=None):
    df=pd.DataFrame(file_list, columns=['file_name'])
    df_ex=df['file_name'].str.split('_', expand=True)
    
    df_ex=df_ex.rename(columns={0:'location',1:'lake_id',2:'name'})
    df_ex['name']=df_ex['name'].apply(lambda x:x[:-4])
    df_all=pd.concat((df,df_ex), axis=1)
    if by=='lake_id':
        return df_all['lake_id']
    elif by=='name':
        return df_all['name']
    else:
        return df_all
postfix='.xlsx'
path='../data/brasil/'
files = [f for f in os.listdir(path) if (os.path.isfile(os.path.join(path, f))&
                                                  (f.endswith(postfix)))]
files
start_st=3
end_st=8
st=[f[start_st:end_st] for f in files]
for s in st:
    print(files[st.index(s)])
    
# type(get_all_lakes(data, by='lake_id'))
# type(get_all_lakes(data, by=None))
# type(get_all_lakes(data, by='name'))
# _get_all_lakes(files, by='lake_id')
# df=_get_all_lakes(files, by=None)

# for index, row in df.iterrows():
#     print(row['file_name'])
#     df=pd.read_excel(path+row['file_name'])
#     break
# # [f[] for f in files] 
# df
# f[:-len(postfix)][-length_id:]
# _get_all_lakes(files, by='name')
# df_coord['lon_tx'].astype(float)/100

br_19069_MACHADINHO.xlsx
br_19126_XINGO.xlsx
br_19060_SEGREDO.xlsx


In [ ]:
#To test what is faster 

import xlrd
import timeit

path='/home/angelica/Documents/py_linux/pylocss/data/brasil/br_19126_XINGO.ods'
# xlrd.open_workbook(filename=path)
%timeit pd.read_excel(path, engine='odf')

In [ ]:
path='/home/angelica/Documents/py_linux/pylocss/data/brasil/br_19126_XINGO.xlsx'
# xlrd.open_workbook(filename=path)
%timeit pd.read_excel(path)